# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [1]:
import pandas as pd #tablas https://pandas.pydata.org/docs/user_guide/pyarrow.html
import numpy as np #estadistica https://numpy.org/doc/1.25/user/index.html#user
import matplotlib.pyplot as plt #gráficos https://matplotlib.org/stable/plot_types/index.html
import seaborn as sns #gráficos https://seaborn.pydata.org/tutorial/regression.html
from sklearn import linear_model #modelo de regresion https://scikit-learn.org/stable/
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from datetime import datetime #https://docs.python.org/3/library/datetime.html
from datetime import timedelta #this is a good time to learn about dates and timestamps. You can find some info here:


#### Open the ` Employee.csv` file and name your dataset `employee`.

In [24]:
# your answer here

employee = pd.read_csv("employee.csv")
employee

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55
5,Eva,Sales,Bachelor,F,associate,2,55
6,Carlos,IT,Master,M,VP,8,70
7,Pedro,IT,Phd,M,associate,7,60
8,Ana,HR,Master,F,VP,8,70


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with groupby and the other way is with pivot tables. Do it both ways. 


In [5]:
# Method 1
# your answer here
employee.groupby(['Department']).agg({'Salary':"mean"})

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


In [12]:
employee.pivot_table(index= ["Department"], values = ['Salary'])

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [11]:
employee.pivot_table(index= ["Department", "Title"], values = ['Salary'])


Salary
Department Title            
HR         VP           70.0
           analyst      32.5
IT         VP           70.0
           analyst      32.5
           associate    60.0
Sales      associate    55.0

#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [20]:
number_employees
employee.pivot_table(index= ["Title"], columns = ["Years"], values = ['Salary'])

Salary                              
Years          1     2     3     4     7     8
Title                                         
VP           NaN   NaN   NaN   NaN   NaN  70.0
analyst     35.0  30.0   NaN  35.0   NaN   NaN
associate    NaN  55.0  55.0   NaN  60.0   NaN

#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [25]:

employee.pivot_table(index= ["Title","Department"], values = ['Salary', "Name"], aggfunc = {'Salary':'mean','Name':pd.Series.nunique})

Name  Salary
Title     Department              
VP        HR             1    70.0
          IT             1    70.0
analyst   HR             2    32.5
          IT             2    32.5
associate IT             1    60.0
          Sales          2    55.0

#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [35]:
# your answer here
df = employee.pivot_table(index= ["Title","Department"], values = ['Salary'], aggfunc = {'mean',"median", "std"}).reset_index()
df

Title Department Salary                 
                          mean median       std
0         VP         HR   70.0   70.0       NaN
1         VP         IT   70.0   70.0       NaN
2    analyst         HR   32.5   32.5  3.535534
3    analyst         IT   32.5   32.5  3.535534
4  associate         IT   60.0   60.0       NaN
5  associate      Sales   55.0   55.0  0.000000

#### Based on your comments, fill the missing values with an appropriate value.

In [42]:
df[('Salary', 'std')] = df[('Salary', 'std')].fillna(0)
df

Title Department Salary                 
                          mean median       std
0         VP         HR   70.0   70.0  0.000000
1         VP         IT   70.0   70.0  0.000000
2    analyst         HR   32.5   32.5  3.535534
3    analyst         IT   32.5   32.5  3.535534
4  associate         IT   60.0   60.0  0.000000
5  associate      Sales   55.0   55.0  0.000000

#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [49]:
employee.pivot_table(index=["Department"], values=["Salary", "Name"], aggfunc={'Salary': 'sum', 'Name': pd.Series.nunique}).reset_index()


,Department,Name,Salary
0,HR,3,135
1,IT,4,195
2,Sales,2,110


#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [52]:
# your answer here
employee.pivot_table(index=["Department", "Title"], values=["Salary", "Years"], aggfunc={'Salary': 'mean', 'Years': "sum"}).reset_index()

,Department,Title,Salary,Years
0,HR,VP,70.0,8
1,HR,analyst,32.5,6
2,IT,VP,70.0,8
3,IT,analyst,32.5,3
4,IT,associate,60.0,7
5,Sales,associate,55.0,5


### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

In [69]:
def mean_noMax(col):
    col = col[col != col.max()].mean()
    return round(col,2)
    
employee.pivot_table(index=['Department'], aggfunc = {'Salary':mean_noMax}).reset_index()

,Department,Salary
0,HR,32.50
1,IT,41.67
